### Code by John Pan (260739619)

In [58]:
#Code by John Pan 260739619
import numpy as np
import statistics as stat

### Question 3 a) BRUTE FORCE

In [86]:
#Get a list of all possible travel routes
from sympy.utilities.iterables import multiset_permutations
a = np.array([0,1,2,3,4,5,6]) #7 cities
city_perms = []
for p in multiset_permutations(a):
    city_perms.append(p)


In [209]:
TSP_insts = [] #store instances of TSP maps
dist = [] #store tour lengths for 1 TSP inst
best_dists = [] #store best tour lengths of all TSP instances

#Get 100 TSP instances (A list of 100 arrays, for which each row is 1 coordinate of a city)
for i in range(0,100):
    TSP_insts.append(np.random.rand(14).reshape(7,2)) #get a TSP instance

#Get distance for each TSP instance
for i in range(0,100):   
    TSP = TSP_insts[i]
    dist = [] #reset to empty for next instance
    
    for u in range(0,len(city_perms)):
        #get distances between each city
        currMap = city_perms[u] #get current permutation
        tmp_dist = 0 #accumulates distance between cities of 1 permutation, reset before next permutation
        
        # sum up the distances between each city of this permutation
        for j in range(0,6):
            a = TSP[currMap[j]]
            b = TSP[currMap[j+1]]
            tmp_dist += np.linalg.norm(a-b)
        
        #save this tour length and then do next possible route
        dist.append(tmp_dist)
    
    #store the min tour length of this instance
    best_dists.append(min(dist))

mean = stat.mean(best_dists)
stdev = stat.stdev(best_dists)
mmin = min(best_dists)
mmax = max(best_dists)

print("brute force mean is: ", mean)
print("brute force standard deviation is: ", stdev)
print("brute force max/min: ",mmax,"/",mmin)
#print(best_dists)

brute force mean is:  1.7456529330587838
brute force standard deviation is:  0.34643956973354434
brute force max/min:  2.764580334472379 / 0.9898952758001994


### Question b) BASELINE AND HILL CLIMB (run below functions first)

In [236]:
#reuse TSP_insts from above
randTour = [] #store the random tour lengths
hillclimb_tours = [] #store the tour lengths obtained using hillclimb
countrand = 0 #count number of times we coincidentally get optimal solution
counthill = 0 #count number of times we get optimal solution using hillclimb

for i in range(0,100):   
    TSP = TSP_insts[i]
    dist = [] #reset to empty for next instance
    mmap = np.random.permutation(7)
    tmp_dist = 0 #reset
    
    # hill-climb section with random tour as start state
    hillclimb_tours.append(hillclimb(TSP,mmap,7)) #RUN BELOW BLOCKS WITH HILLCLIMB FUNCTION FIRST
    if abs(hillclimb_tours[i]-min(best_dists)) < 0.1: #check if we got an optimal length. Use tolerance of 0.001
        counthill += 1
    #hill-climb section end
    
    # baseline section, with random tour for each instance
    
    # calculate distance of this tour of 7 cities
    for j in range(0,6):
        a = TSP[mmap[j]]
        b = TSP[mmap[j+1]]
        tmp_dist += np.linalg.norm(a-b)
    
    if abs(tmp_dist - min(best_dists)) < 0.1: #check if we got an optimal length. Use tolerance of 0.001
        countrand += 1
        
    randTour.append(tmp_dist) #save the tour length of this instance

# baseline stats
rmean = stat.mean(randTour)
rstdev = stat.stdev(randTour)
rmin = min(randTour)
rmax = max(randTour)

print("baseline mean is: ", rmean)
print("baseline standard deviation is: ", rstdev)
print("baseline max/min: ",rmax,"/",rmin)
print("baseline: number of times a random tour is optimal: ", countrand)

#hill climb stats
hmean = stat.mean(hillclimb_tours)
hstdev = stat.stdev(hillclimb_tours)
hmin = min(hillclimb_tours)
hmax = max(hillclimb_tours)

print("hill climb algo mean is: ", hmean)
print("hill climb algo standard deviation is: ", hstdev)
print("hill climb algo max/min: ",hmax,"/",hmin)
print("hill climb: number of times an optimal tour is found: ", counthill)

baseline mean is:  3.1313454293676983
baseline standard deviation is:  0.6686589956104334
baseline max/min:  4.94460987438775 / 2.048955449197502
baseline: number of times a random tour is optimal:  0
hill climb algo mean is:  2.3816802923278755
hill climb algo standard deviation is:  0.53279206742532
hill climb algo max/min:  3.781644888003811 / 0.9898952758001995
hill climb: number of times an optimal tour is found:  1


### Q3 C) HILL CLIMB CODE

In [159]:
#generate neighbours function
import copy as cp
def neighbours(currPath, ncity):
    neighboursList = []
    for i in range(0,ncity-1):
        tmpPath = cp.deepcopy(currPath)
        tmpPath[[i,i+1]] = tmpPath[[i+1,i]]
        neighboursList.append(tmpPath)
    return neighboursList

#calculates the cost 
def pathCost(TSP,path,ncity):
    tmp_dist = 0
    for j in range(0,ncity-1):
        a = TSP[path[j]] #gets the row of city represented by coordinates x = col1, y =col2
        b = TSP[path[j+1]]
        tmp_dist += np.linalg.norm(a-b)
    return tmp_dist

In [160]:
def hillclimb(TSP,randpath,ncity):
    E = pathCost(TSP,randpath,ncity) #get initial tour length
    X = randpath #get initial tour 
    neighbours_cost = [] #store neighbours tour lengths
    inf = 0
    
    while inf == 0: #infinite loop
        neighbs = neighbours(X,ncity) #get the neighbours, list of 6 maps, which are 1d arrays
        #get neighbours tour lengths
        for i in range(0,ncity-1): #there are six possible neighbours by switch 2 adjacent cities for a map of 7 cities
            neighbours_cost.append(pathCost(TSP,neighbs[i],ncity))

        if min(neighbours_cost) >= E: #already have shortest length
            return E #we return the current shortest tour directly
        else: #update to new shortest tour
            E = min(neighbours_cost)
            X = neighbs[neighbours_cost.index(min(neighbours_cost))] 
            neighbours_cost.clear()
    
    
    

### 100 CITIES

In [172]:
#reuse TSP_insts from above
hTSP_insts = []
hrandTour = [] #store the random tour lengths
hhillclimb_tours = [] #store the tour lengths obtained using hillclimb

#Get 100 TSP instances (A list of 100 arrays, for which each row is 1 coordinate of a city)
for i in range(0,100):
    hTSP_insts.append(np.random.rand(200).reshape(100,2)) #get a TSP instance of 100 cities

for i in range(0,100):   
    hTSP = hTSP_insts[i]
    dist = [] #reset to empty for next instance
    hmap = np.random.permutation(100)
    tmp_dist = 0 #reset
    
    # hill-climb section with random tour as start state
    hhillclimb_tours.append(hillclimb(hTSP,hmap,100)) #RUN BLOCKS WITH HILLCLIMB FUNCTION FIRST
    
    # baseline section, with random tour for each instance
    
    # calculate distance of this tour of 100 cities
    for j in range(0,99):
        a = hTSP[hmap[j]]
        b = hTSP[hmap[j+1]]
        tmp_dist += np.linalg.norm(a-b)
        
    hrandTour.append(tmp_dist) #save the tour length of this instance

# stats for 100 cities
hrmean = stat.mean(hrandTour)
hrstdev = stat.stdev(hrandTour)
hrmin = min(hrandTour)
hrmax = max(hrandTour)

print("baseline mean is: ", hrmean)
print("baseline standard deviation is: ", hrstdev)
print("baseline max/min: ",hrmax,"/",hrmin)

#hill climb stats
hhmean = stat.mean(hhillclimb_tours)
hhstdev = stat.stdev(hhillclimb_tours)
hhmin = min(hhillclimb_tours)
hhmax = max(hhillclimb_tours)

print("hill climb algo mean is: ", hhmean)
print("hill climb algo standard deviation is: ", hhstdev)
print("hill climb algo max/min: ",hhmax,"/",hhmin)

baseline mean is:  52.031883786960016
baseline standard deviation is:  2.6007175448943607
baseline max/min:  58.91093600885729 / 45.27802023846601
hill climb algo mean is:  39.13722423606931
hill climb algo standard deviation is:  1.9447130836957343
hill climb algo max/min:  44.92750949113765 / 34.397808388287345
